# Gestenerkennung (Punch/Flex)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import random

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

!python --version
print("NumPy: " + np.__version__)
print("Keras: " + tf.keras.__version__)
print("TensorFlow: " + tf.__version__)

# Teil 1: Modellerstellung

### Datensatz importieren

In [ ]:
# Daten importieren

#df = pd.read_csv("../data/punch.csv")
df = pd.read_csv("../data/flex.csv")

### Daten anzeigen/visualisieren

In [ ]:
df
#df.shape
#df.min()
#df.max()
#df.describe()

In [ ]:
# Plot der Beschleuniger-Daten

plt.figure(figsize=(14,8))
plt.plot(df['aX'], 'g', label='x', linestyle='solid')
plt.plot(df['aY'], 'b', label='y', linestyle='solid')
plt.plot(df['aZ'], 'r', label='z', linestyle='solid')

for i in range(9):
  plt.vlines((i+1)*104, -4000, 4000, linestyle="dashed", color='gray')

plt.title("Beschleunigung");
plt.xlabel("Sample");
plt.ylabel("Beschleunigung (mG)");
plt.legend();

In [ ]:
# Plot der Gyroskop-Daten

plt.figure(figsize=(14,8))
plt.plot(df['gX'], 'g', label='x', linestyle='solid')
plt.plot(df['gY'], 'b', label='y', linestyle='solid')
plt.plot(df['gZ'], 'r', label='z', linestyle='solid')
plt.title("Gyroskop")
plt.xlabel("Sample")
plt.ylabel("Gyroskop (Grad/Sek)")
plt.legend();

## Trainingsdaten vorbereiten

In [ ]:
# Zufallszahlengenerator fixieren (für reproduzierbare Ergebnisse)
SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)

GESTURES = [
    "punch",
    "flex",
]

SAMPLES_PER_GESTURE = 104         # muss mit Wert in IMU_Capture (Arduino) übereinstimmen

NUM_GESTURES = len(GESTURES)

# One-Hot-Kodierte Matrix für Labels
ONE_HOT_ENCODED_GESTURES = np.eye(NUM_GESTURES)

inputs = []
outputs = []

# lese jede einzelne CSV-Datei ein und erzeuge die Trainingsdaten und Label
# Serialisieren der Daten (6 Sensorkanäle á SAMPLES_PER_GESTURE)
# Normalisierung der Daten
for gesture_index in range(NUM_GESTURES):
    
  gesture = GESTURES[gesture_index]
  print(f"Verarbeite Index %d für Geste '%s'." % (gesture_index, gesture), end=" ")
  
  output = ONE_HOT_ENCODED_GESTURES[gesture_index]
  
  # Einlesen der CSV-Daten
  df = pd.read_csv("../data/" + gesture + ".csv")
  
  # Berechnung der Anzahl der Aufnahmen pro Geste und Datei
  # Anzahl aller Samples/Samples pro Geste
  num_recordings = int(df.shape[0] / SAMPLES_PER_GESTURE)
  
  print("Im Datensatz wurden %d Aufzeichnungen für die Geste '%s' gefunden." % (num_recordings, gesture))
  
  # Daten normalisieren und serialisieren
  for i in range(num_recordings):
    tensor = []
    for j in range(SAMPLES_PER_GESTURE):
      index = i * SAMPLES_PER_GESTURE + j
      # Normalisierung der Daten (0...1)
      # - Beschleunigung: -4000 to +4000
      # - Gyroskop: -2000 to +2000
      tensor += [
          (df['aX'][index] + 4000) / 8000,
          (df['aY'][index] + 4000) / 8000,
          (df['aZ'][index] + 4000) / 8000,
          (df['gX'][index] + 8000) / 16000,
          (df['gY'][index] + 8000) / 16000,
          (df['gZ'][index] + 8000) / 16000
      ]

    inputs.append(tensor)
    outputs.append(output)

# Python-Listen in ein NumPy-Array konvertieren
inputs  = np.array(inputs)
outputs = np.array(outputs)

### Daten randomisieren

Daten werden nun per Zufall gemischt: 60% Trainingsdaten, 20% Validierungsdaten und 20% Testdaten

- Trainingsdaten: für das Training des Modells
- Validierungsdaten: Berechnung der Performance während des Trainings
- Testdaten: Test des Modells nach dem Training

In [ ]:
num_inputs = len(inputs)
randomize = np.arange(num_inputs)
np.random.shuffle(randomize)

inputs = inputs[randomize]
outputs = outputs[randomize]

TRAIN_SPLIT = int(0.6 * num_inputs)
TEST_SPLIT = int(0.2 * num_inputs + TRAIN_SPLIT)

inputs_train, inputs_test, inputs_validate = np.split(inputs, [TRAIN_SPLIT, TEST_SPLIT])
outputs_train, outputs_test, outputs_validate = np.split(outputs, [TRAIN_SPLIT, TEST_SPLIT])

## Entwurf des neuronalen Netzes

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Input(shape=[624,]))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(15, activation='relu'))
model.add(tf.keras.layers.Dense(NUM_GESTURES, activation='softmax'))

In [ ]:
# Kompilierung
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Zusammenfassung
model.summary()

## Training des neuronalen Netzes

In [ ]:
history = model.fit(inputs_train, outputs_train, epochs=100, batch_size=1, verbose=1, validation_data=(inputs_validate, outputs_validate))

In [ ]:
# Trainingsverlauf grafisch darstellen

epochs = range(1, len(history.history['val_loss']) + 1)

plt.figure(figsize=(8,7))
plt.plot(epochs, history.history['val_loss'])
plt.xlabel('Epoche')
plt.ylabel('Fehler')
plt.grid(True)

## Test

In [ ]:
predictions = model.predict(inputs_test)

print("Vorhersage =\n", np.round(predictions, decimals=3))
print("Label =\n", outputs_test)

# Teil 2: Portierung auf Mikrocontroller

### Konvertierung in ein TensorFlow lite-Modell

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimazations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Modell speichern
# Keras-Modell speichern
open('gesture_model.tflite', 'wb').write(tflite_model);